# Masterclass

### Importieren Bibliotheken

An dieser Stelle werden die Bibliotheken importiert, die für die Datenanalyse notwendig sind.
Wenn alles fertig importiert ist, wird es Dir vom Programm angezeigt. Dann kannst Du weitermachen.

In [ ]:
import ROOT
import uproot
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
import math
import requests
print("Alles geladen!")

### Gruppenauswahl

Hier sollst Du deine Gruppennummer eingeben.

In [ ]:
gruppe=11
if gruppe>10 or gruppe<0 or gruppe!=int(gruppe):
    print("Du solltest eine Gruppennummer zwischen 1 und 10 haben!")

Hier wird die richtige Datei Heruntergeladen.

Das kann dauern, bitte nur einmal ausführen!
Das Programm zeigt Dir an, wenn es die entsprechende Datei fertig geladen hat.

In [ ]:
urls=["https://cernbox.cern.ch/index.php/s/QS4iT8gIv8xFzNu/download", #1
      "https://cernbox.cern.ch/index.php/s/ACPAHhinDXVUrvu/download", #2
      "https://cernbox.cern.ch/index.php/s/BGCTRtUUyMrFzQN/download", #3
      "https://cernbox.cern.ch/index.php/s/rvstLklNQTZM7eG/download", #4
      "https://cernbox.cern.ch/index.php/s/fmUytQBMwNuUYQq/download", #5
      "https://cernbox.cern.ch/index.php/s/G03MHIxqZjYbK6u/download", #6
      "https://cernbox.cern.ch/index.php/s/0mSb3DhjX2FqK3o/download", #7
      "https://cernbox.cern.ch/index.php/s/zhzUE2HwUX82pOm/download", #8
      "https://cernbox.cern.ch/index.php/s/rnyG9b2O4CZo7YR/download", #9
      "https://cernbox.cern.ch/index.php/s/CK0zEsHhqgXA0EB/download", #10
     ]
urlsWS=["https://cernbox.cern.ch/index.php/s/L3HseMtKZWKkwp4/download", #1
      "https://cernbox.cern.ch/index.php/s/zGFePJmF5op4htU/download", #2
      "https://cernbox.cern.ch/index.php/s/ZL06Lrtnp9ptlwm/download", #3
      "https://cernbox.cern.ch/index.php/s/oGArqPD6TuYlEkd/download", #4
      "https://cernbox.cern.ch/index.php/s/bEqEkecN6vDCzZv/download", #5
      "https://cernbox.cern.ch/index.php/s/96guIz3nSK6r9Vs/download", #6
      "https://cernbox.cern.ch/index.php/s/Tg2BvkUod7RzvbC/download", #7
      "https://cernbox.cern.ch/index.php/s/e4SAM5l5MNRFvgK/download", #8
      "https://cernbox.cern.ch/index.php/s/KMH4vI9gwUpZGth/download", #9
      "https://cernbox.cern.ch/index.php/s/Rev3W2fIbx9QEgg/download", #10
     ]
myfile = requests.get(urls[gruppe-1])
open('Masterclass{}.root'.format(gruppe), 'wb').write(myfile.content)
myfile = requests.get(urlsWS[gruppe-1])
open('Masterclass{}WS.root'.format(gruppe), 'wb').write(myfile.content)
print("Wir haben alle Daten heruntergeladen, es kann los gehen!")

### Erstellen von Dataframes

Hier werden die Daten ausgelesen und für die Analyse in Dataframes gespeichert, damit Filter angewendet und Histogramme erstellt werden können.

In [ ]:
df = uproot.open("Masterclass{}.root".format(gruppe))["DecayTree"].arrays(library="pd")
f = ROOT.TFile.Open("Masterclass{}.root".format(gruppe))
tree1 = f.Get("DecayTree")

gf = uproot.open("Masterclass{}WS.root".format(gruppe))["DecayTree"].arrays(library="pd")
g = ROOT.TFile.Open("Masterclass{}WS.root".format(gruppe))
tree2 = g.Get("DecayTree")

### Aufgabe 1: $\Xi_c^+$
#### Urprüngliches Signal

Zunächst wird kannst Du hier die ursprünglichen, ungefilterten Daten in einem Histogramm darstellen lassen.
Da Du in dieser Aufgabe ein möglichst reines Signal von $\Xi_c^+$ erzeugen sollst, rechnet die nächste Zelle zusätzlich einen Wert für diese Reinheit vom ungefilterten Datensatz aus.

In [ ]:
xmin = 2420
xmax = 2510
ROOT.gStyle.SetOptStat(0) #Entfernen HistLegende

fit1 = ROOT.TF1('fit1','[0] + x*[1] + [3] * exp(-(x - [4])^2/(2*[2]*[2]))+ [5] * exp(-(x - [7])^2/(2*[6]*[6]))', xmin, xmax) #Fitfunktion
fit1.SetParameter(4, 2468) #Parameter für Gauss Tendenz setzen Mean
fit1.SetParameter(3,1) #Parameter für Gauss Tendenz setzen existenz
fit1.SetParameter(5,1) #existenz Gauss2
fit1.SetParameter(7, 2468) #Mean Gauss2

h11 = ROOT.TH1F("h11","Kombination der Teilchen", 100, 2420, 2510)
c11 = ROOT.TCanvas("myCanvasName","Titel",800,600)
tree1.Draw("lab1_M>>h11")
h11.SetLineColor(ROOT.kRed)
h11.Draw("MIN0")

h11.Fit("fit1", "RQ")
a0 = fit1.GetParameter("p0")
a1 = fit1.GetParameter("p1")
a2 = fit1.GetParameter("p2")
a3 = fit1.GetParameter("p3")
a4 = fit1.GetParameter("p4")
a5 = fit1.GetParameter("p5")
a6 = fit1.GetParameter("p6")
a7 = fit1.GetParameter("p7")

sighin = ROOT.TF1("sighin", "{a0}+x*{a2}+{a3} * exp(-(x - {a4})^2/(2*{a2}*{a2}))+ {a5} * exp(-(x - {a7})^2/(2*{a6}*{a6}))".format(a0=a0,a1=a1,a2=a2,a3=a3,a4=a4,a5=a5,a6=a6,a7=a7), xmin, xmax)
sig1 = ROOT.TF1("sig1", "{a3} * exp(-(x - {a4})^2/(2*{a2}*{a2}))+ {a5} * exp(-(x - {a7})^2/(2*{a6}*{a6}))".format(a2=a2,a3=a3,a4=a4,a5=a5,a6=a6,a7=a7), xmin, xmax)
sig1.SetLineWidth(4)
bac1 = ROOT.TF1("bac1", "{a0}+ x*{a1}".format(a0=a0,a1=a1), xmin, xmax)
bac1.SetLineColor(1)
bac1.SetLineStyle(4)
sig1.Draw("same")
bac1.Draw("same")

SigHin = sighin.Integral(a4 - 2 * abs(a2), a4 + 2 * abs(a2)) 
Signal = sig1.Integral(a4 - 2*abs(a2), a4 + 2*abs(a2))
Purity = Signal/(math.sqrt(SigHin))

print("Einträge Signal:", round(Signal,2))
print("Einträge Signal + Untergrund:", round(SigHin,2))
print("Reinheit des Signals:", round(Purity,2))
print("Breite des Signals:", round(abs(a2),2),"MeV")
print(" ")

%jsroot
c11.Draw()
h11.GetXaxis().SetTitle("m(p K^{-} #pi^{+}) [MeV]")
h11.GetYaxis().SetTitle("Eintr#ddot{a}ge pro 1.1MeV")
c11.SetLeftMargin(0.12)
leg=ROOT.TLegend(0.8,0.7,0.99,0.99)
leg.AddEntry(h11,"Ungefilterte Daten","l")
leg.AddEntry(bac1,"Untergrund","1")
leg.AddEntry(sig1,"Ungefiltertes Signal","l")
leg.Draw()

#### Schieberegler

In der nächsten Zelle darfst Du zum ersten Mal selbst arbeiten, hier beginnt die Datenanalyse! Du sollst ein möglichst sauberes Signal von $\Xi_c^+$ erhalten. Dafür wendest Du Filter auf die Daten an, um den Hintergrund zu reduzieren.

Das Signal für $\Xi_c^+$ soll im orangefarbenen Histogramm bestimmt werden. Mit jedem angewandten Filter updated es sich, behält aber das ursprüngliche ungefilterte Signal im Hintergrund bei. In den blauen Histogrammen siehst Du die Werteverteilung der Filter. Daran kannst Du erahnen, welche Filterung sinnvoll ist und welche eher nicht.

Es kann zunächst einen kurzen Moment dauern, bis die Histogramme erstellt sind, aber dann kann es direkt losgehen. Mithilfe der Schieberegler können die einzelnen Filter angepasst und so die Daten zugeschnitten werden. Eine Beschreibung, was welcher Filter bedeutet, findest du auf dem Aufgabenblatt.

Wenn Du einen Schieberegler verstellt hast, wartest Du am besten kurz (5-10s), bis der zuletzt angewendete Filter von den Histogrammen übernommen wurde.

In [ ]:
%matplotlib widget

fig = plt.figure()
fig.set_size_inches(10, 5)
fig.canvas.header_visible = False

ax = plt.subplot2grid((7,16),(1,1),colspan = 7, rowspan = 5)
bx = plt.subplot2grid((7,16),(1,9),colspan = 3, rowspan = 2)
dx = plt.subplot2grid((7,16),(4,9),colspan = 3, rowspan = 2)
cx = plt.subplot2grid((7,16),(1,13),colspan = 3, rowspan = 2)
ex = plt.subplot2grid((7,16),(4,13),colspan = 3, rowspan = 2)
plt.tight_layout()

print("Eine kurze Erinnerung: Ist der Slider auf der linken Seite, wird wenig gefiltert, rechts filtert er stark.")
lb1 = widgets.Label("Filter 1: $P(e^-)$ Wahrscheinlichkeit, dass ein Teilchen ein Elektron ist")
lb2 = widgets.Label("Filter 2: $P(e^-)$ Wahrscheinlichkeit, dass ein Teilchen ein Proton ist")
lb3 = widgets.Label("Filter 3: $\chi^2_{Stoß}$ Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt")
lb4 = widgets.Label("Filter 4: $\chi^2_{Flug}$ Qualität der Flugdistanz bezüglich des Kollisionspunkts")
display(lb1, lb2, lb3, lb4)

@widgets.interact(Cut1=widgets.FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.1 $P(e^-)$",continuous_update = False), 
                  Cut2=widgets.FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.2 $P(p)$",continuous_update = False), 
                  Cut3=widgets.FloatSlider(value=0,min = 0, max = 11, step = 0.1, description = "Ftr.3 $\chi^2_{Stoß}$",continuous_update = False), 
                  Cut4=widgets.IntSlider(min = 0, max = 500, step = 2, description = "Ftr.4 $\chi^2_{Fl}$",continuous_update = False))
def update(Cut1, Cut2=0, Cut3=0, Cut4=0):
    
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    
    ax.set_title("Kombination von $p$, $K^-$ und $\pi^+$")
    ax.set_ylabel("Einträge pro 1.1MeV")
    ax.set_xlabel("$m(pK^-\pi^+)$ [$MeV$]")
    
    bx.set_title("")
    bx.set_ylabel("Einträge")
    bx.set_xlabel("Filter 1: Elektron")
    bx.set_yscale("log")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Filter 2: Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("Filter 3: Stoßparameter")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Filter 4: Flugdistanz")
    
    ax.hist(df["lab1_M"], bins=100, range =(2420,2510), color = "brown")
    ax.hist(df["lab1_M"][df["lab2_ProbNNe"]>Cut1][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(2420,2510), color = "darkorange")
    bx.hist(df["lab2_ProbNNe"][df["lab2_ProbNNe"]>Cut1], bins=100, range =(0,1))
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))

#### Festlegung der Filter

In die nächste Zelle sollst Du die Werte für die Filter eintragen, die Du in dem vorherigen Teil mit den Schiebereglern erörtert hast. Ersetze dafür die drei Rauten ### in der Zelle mit den Zahlenwerten und nutze Punkte statt Kommas, wenn Du Dezimalzahlen schreibst. Die Zelle gibt Dir eine Mitteilung, wenn die Filter übernommen sind.

In [ ]:
cut11 = ROOT.TCut("lab2_ProbNNe>###") #Einfügen Filter1
cut12 = ROOT.TCut("lab2_ProbNNp>###") #Einfügen Filter2
cut13 = ROOT.TCut("lab1_IPCHI2_OWNPV<(11-###)") #Einfügen Filter3
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>###") #Einfügen Filter4
cut10 = ROOT.TCut(cut11+cut12+cut13+cut14)

print("Werte der Filter sind übernommen!")

#### Berechnung der Reinheit

Wenn Du Werte für die Filter angegeben hast und die Zelle danach ausgeführt hast, kannst Du die nächste Zelle ausführen. Hier wird ein Histogramm der Daten mit den von Dir bestimmten Filtern erstellt, was anschließend eine Ausgleichskurve kriegt. Außerdem berechnet das Programm die Anzahl der Einträge von Signal und Hintergrund und gibt neben der Breite des Signals auch dessen Reinheit an.

In [ ]:
xmin = 2420
xmax = 2510
ROOT.gStyle.SetOptStat(0)

#Erstellen Fitfunktion:
f1 = ROOT.TF1('f1','[0] + x*[1] + [3] * exp(-(x - [4])^2/(2*[2]*[2]))+ [5] * exp(-(x - [7])^2/(2*[6]*[6]))', xmin, xmax)
f1.SetParameter(4, 2468)
f1.SetParameter(3,1) 
f1.SetParameter(5,1) 
f1.SetParameter(7, 2468)

#Erstellen und Füllen der Histogramme
h1 = ROOT.TH1F("h1", "Kombination der Teilchen", 100, xmin, xmax) #Hist mit
h2 = ROOT.TH1F("h2", "Kombination der Teilchen", 100, xmin, xmax) #Hist ohne
tree1.Draw("lab1_M>>h1",cut10) #Füllen Hist mit
tree1.Draw("lab1_M>>h2") #Füllen Hist ohne
c1 = ROOT.TCanvas("myCanvasName","Titel",800,600) #Erstellen Canvas
h2.SetLineColor(ROOT.kRed)
h2.Draw("MIN0") #Hist ohne zeichnen (y-Achse ab 0)
h1.Draw("same") #Hist mit zeichnen in gleiches Canvas

#erste Fits, Fitparameter für Integrale nehmen
f1.SetLineColor(ROOT.kBlue-6)
h1.Fit("f1","RQ") #Hist mC fitten
p10 = f1.GetParameter("p0") #getParForBG#Y-Intersect
p11 = f1.GetParameter("p1") #GetParForBG#slope
p22 = f1.GetParameter("p2") #GetParForSignal #sigma
p23 = f1.GetParameter("p3") #Signal
p24 = f1.GetParameter("p4") #Signal #Mean
p25 = f1.GetParameter("p5") #Signal 
p26 = f1.GetParameter("p6") #Signal #sigma
p27 = f1.GetParameter("p7") #Signal #mean

#Bestimmen der Funktione für Integrale, kosmetische Anpassungen Fits
f21 = ROOT.TF1("f21", "{p10}+x*{p11}".format(p10=p10, p11=p11), 0, xmax) #BGFunctionCut
f21.SetLineColor(1) #BGCut
f21.SetLineStyle(4) #BGCut
f21.Draw("same") #Draw BGCut
f22 = ROOT.TF1("f22", "{p23} * exp(-(x - {p24})^2/(2*{p22}*{p22}))+ {p25} * exp(-(x - {p27})^2/(2*{p26}*{p26}))".format(p22=p22,p23=p23,p24=p24,p25=p25,p26=p26,p27=p27), xmin,xmax) #only SignalPeak Fit
f23 = ROOT.TF1("f23", "{p10}+x*{p11} +{p23} * exp(-(x - {p24})^2/(2*{p22}*{p22}))+ {p25} * exp(-(x - {p27})^2/(2*{p26}*{p26}))".format(p10=p10,p11=p11,p22=p22,p23=p23,p24=p24,p25=p25,p26=p26,p27=p27), xmin,xmax)
f22.SetLineColor(ROOT.kBlue+2)
f22.SetLineWidth(3)
f22.Draw("same") #Draw SignalPeak
SB = f23.Integral(p24 - 2 * abs(p22), p24 + 2 * abs(p22)) #integrate Sign+Bg in signalregion (mean+-2sigma)
Signal = f22.Integral(p24 - 2*abs(p22), p24 + 2*abs(p22)) #integrate signal in signalregion ("")
f23.SetLineColor(1)
f23.SetLineStyle(4)
#f23.Draw("same")

#Variablen für die
Sigma = abs(p22)
sqrtSB= math.sqrt(SB)
Purity = Signal/sqrtSB
print("Einträge Signal:", round(Signal,2))
print("Einträge Signal + Untergrund:", round(SB,2))
print("Reinheit des Signals:", round(Purity,2))
print("Breite des Signals:", round(Sigma,2), "MeV")
print(" ")

f1.SetParameter(4, 2468)
f1.SetParameter(3,1) 
f1.SetParameter(5,1) 
f1.SetParameter(7, 2468) 
f1.SetLineColor(ROOT.kRed)
h2.Fit("f1","RQ") #ursprüngliches Signal
p0 = f1.GetParameter("p0")
p1 = f1.GetParameter("p1")

f3 = ROOT.TF1("f3", "{p0}+x*{p1}".format(p0=p0, p1=p1), 0, xmax) #ursprünglicher Hintergrund
f3.SetLineColor(1) #ursprünglicher Hintergrund
f3.SetLineStyle(4) #ursprünglicher Hintergrund
f3.Draw("same")
%jsroot
c1.Draw()
h2.GetXaxis().SetTitle("m(p K^{-} #pi^{+}) [MeV]")
h2.GetYaxis().SetTitle("Eintr#ddot{a}ge pro 1.1MeV")
c1.SetLeftMargin(0.12)
leg=ROOT.TLegend(0.8,0.7,0.99,0.99)
leg.AddEntry(h2,"Ungefilterte Daten","l")
leg.AddEntry(h1,"Gefilterte Daten","l")
leg.AddEntry(f22,"Gefiltertes Signal","l")
leg.Draw()

### Aufgabe 2: $\Omega_c^0$
Hier sollst Du Besonderheiten von $\Omega_c^0$ finden! Dafür kombinieren wir $\Xi_c^+$ einmal mit $K^-$, einmal mit $K^+$.

#### Schieberegler
In der nächsten Zelle sollst Du wieder mithilfe der Schiebregler arbeiten. Um die angeregten Zustände zu finden, soll der Untergrund wieder rausgefiltert werden. Die blauen Histogramme zeigen Dir die Werteverteilungen der Filter an, in den grünen Histogrammen sind die beiden Kombinationen von $\Xi_c^+$ mit den Kaons.


Ist der Schieberegler links, wird wenig Filter angewandt. Rechts wird viel gefiltert. Am besten wartest Du auch hier immer kurz (5-10s) bis der zuletzt angewendete Filter von den Histogrammen übernommen wurde.

In [ ]:
%matplotlib widget

df = df.assign(Xic= df["lab0_M"] * 0 + 2467.89)
gf = gf.assign(Xic= gf["lab0_M"] * 0 + 2467.89)


fig = plt.figure()
fig.set_size_inches(10, 6)
fig.canvas.header_visible = False

ax = plt.subplot2grid((13,14),(1,1),colspan = 5, rowspan = 5)
bx = plt.subplot2grid((13,14),(7,1),colspan = 5, rowspan = 5)
cx = plt.subplot2grid((13,14),(1,7),colspan = 3, rowspan = 3)
ex = plt.subplot2grid((13,14),(5,7),colspan = 3, rowspan = 3)
gx = plt.subplot2grid((13,14),(9,7),colspan = 3, rowspan = 3)
dx = plt.subplot2grid((13,14),(1,11),colspan = 3, rowspan = 3)
fx = plt.subplot2grid((13,14),(5,11),colspan = 3, rowspan = 3)
hx = plt.subplot2grid((13,14),(9,11),colspan = 3, rowspan = 3)
plt.tight_layout()

#print("Bedingung 2: Wahrscheinlichkeit, dass ein Teilchen ein Proton ist")
#print("Bedingung 3: Qualität des Stoßparameters zwischen Teilchen und Kollisionspunkt")
#print("Bedingung 4: Qualität der Flugdistanz bezüglich des Kollisionspunkts")
#print("Bedingung 5: Qualität des Fits mit erwarteten Massenhypothesen (= DTF)")
#print("Bedingung 6: Wahrscheinlichkeit, dass ein Teilchen ein Kaon ist")
#print("Bedingung 7: Größe des Transversalimpulses")
print("Eine kurze Erinnerung: Ist der Slider auf der linken Seite, wird wenig gefiltert, rechts filtert er stark.")
lb5 = widgets.Label("Filter 5: $\chi_{DTF}^{2}$ Qualität des Fits mit erwarteten Massenhypothesen")
lb6 = widgets.Label("Filter 6: $P(K)$ Wahrscheinlichkeit, dass zugefügtes Teilchen ein Kaon ist")
lb7 = widgets.Label("Filter 7: $p_{T}$ Größe des Transversalimpulses")
display(lb2, lb3, lb4, lb5, lb6, lb7)

@widgets.interact(Cut2=widgets.FloatSlider(min = 0, max = 1, step = 0.05, description = "Ftr.2 $P(p)$",continuous_update = False), 
                  Cut3=widgets.FloatSlider(value=0,min = 0, max = 11, step = 0.1, description = "Ftr.3 $\chi^2_{Stoß}$",continuous_update = False), 
                  Cut4=widgets.IntSlider(min = 0, max = 500, step = 2, description = "Ftr.4 $\chi^2_{Fl}$",continuous_update = False),
                  Cut5=widgets.FloatSlider(min = 0, max = 30, step = 0.1, description = "Ftr.5 $\chi^2_{DTF}$",continuous_update = False), 
                  Cut6=widgets.FloatSlider(value=0,min = 0, max = 1, step = 0.01, description = "Ftr.6 $P(K^-)$",continuous_update = False), 
                  Cut7=widgets.IntSlider(min = 0, max = 10000, step = 20, description = "Ftr.7 $p_T$",continuous_update = False))
def update(Cut2=0, Cut3=0, Cut4=0, Cut5=0, Cut6=0, Cut7=0):
    ax.cla()
    bx.cla()
    cx.cla()
    dx.cla()
    ex.cla()
    fx.cla()
    gx.cla()
    hx.cla()

    
    ax.set_title("Kombination $\Xi_c^+$ mit $K^-$")
    ax.set_ylabel("Einträge pro 2.2MeV")
    ax.set_xlabel("$m(\Xi_c^+ K^-)$ [$MeV$]")
    
    bx.set_title("Kombination $\Xi_c^+$ mit $K^+$")
    bx.set_ylabel("Einträge pro 2.2MeV")
    bx.set_xlabel("$m(\Xi_c^+ K^+)$ [$MeV$]")
    
    cx.set_title("")
    cx.set_ylabel("Einträge")
    cx.set_xlabel("Filter 2: Proton")
    
    dx.set_title("")
    dx.set_ylabel("Einträge")
    dx.set_xlabel("Filter 3: Stoßparameter")
    dx.set_yscale("log")
    
    ex.set_title("")
    ex.set_ylabel("Einträge")
    ex.set_xlabel("Filter 4: Flugdistanz")

    fx.set_title("")
    fx.set_ylabel("Einträge")
    fx.set_xlabel("Filter 5: Massenhypothese")

    gx.set_title("")
    gx.set_ylabel("Einträge")
    gx.set_xlabel("Filter 6: Kaon")
    
    hx.set_title("")
    hx.set_ylabel("Einträge")
    hx.set_xlabel("Filter 7: Transversalimpuls")
        
    ax.hist(df["lab0_M"]-df["lab1_M"]+df["Xic"][df["lab2_ProbNNp"]>Cut2][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)][df["lab1_FDCHI2_OWNPV"]>Cut4][df["lab0_CHI2NDOF_DTF_Xic"]<(30-Cut5)][df["lab3_ProbNNk"]>Cut6][df["lab0_PT"]>Cut7], bins=100, range =(2980,3150), color = "green")
    bx.hist(gf["lab0_M"]-gf["lab1_M"]+gf["Xic"][gf["lab2_ProbNNp"]>Cut2][gf["lab1_IPCHI2_OWNPV"]<(11-Cut3)][gf["lab1_FDCHI2_OWNPV"]>Cut4][gf["lab0_CHI2NDOF_DTF_Xic"]<(30-Cut5)][gf["lab3_ProbNNk"]>Cut6][gf["lab0_PT"]>Cut7], bins=100, range =(2980,3150), color = "darkgreen")
    cx.hist(df["lab2_ProbNNp"][df["lab2_ProbNNp"]>Cut2], bins=100, range =(0,1))
    dx.hist(df["lab1_IPCHI2_OWNPV"][df["lab1_IPCHI2_OWNPV"]<(11-Cut3)], bins=100, range =(0,11))
    ex.hist(df["lab1_FDCHI2_OWNPV"][df["lab1_FDCHI2_OWNPV"]>Cut4], bins=100, range =(0,500))
    fx.hist(df["lab0_CHI2NDOF_DTF_Xic"][df["lab0_CHI2NDOF_DTF_Xic"]<(30-Cut5)], bins=100, range =(0,30))
    gx.hist(df["lab3_ProbNNk"][df["lab3_ProbNNk"]>Cut6], bins=100, range =(0,1))
    hx.hist(df["lab0_PT"][df["lab0_PT"]>Cut7], bins=100, range =(0,20000))

#### Festlegung der Filter

In die nächste Zelle sollst Du die Werte für die Filter eintragen, die Du in dem vorherigen Teil mit den Schiebereglern erörtert hast. Ersetze dafür die drei Rauten ### in der Zelle mit den Zahlenwerten und nutze Punkte statt Kommas, wenn Du Dezimalzahlen schreibst. Die Zelle gibt Dir eine Mitteilung, wenn die Filter übernommen sind.

In [ ]:
#Hier die Updates der Filter 2-4 eintragen
cut12 = ROOT.TCut("lab2_ProbNNp>###") #Filter 2
cut13 = ROOT.TCut("log(lab1_IPCHI2_OWNPV)<(11-###)") #Filter 3
cut14 = ROOT.TCut("lab1_FDCHI2_OWNPV>###") #Filter 4
cut10 = ROOT.TCut(cut12+cut13+cut14)

#Hier die Werte für Filter 5-7 eintragen
cut21 = ROOT.TCut("lab0_CHI2NDOF_DTF_Xic<(30-###)") #Filter 5
cut22 = ROOT.TCut("lab3_ProbNNk>###") #Filter 6
cut23 = ROOT.TCut("lab0_PT>###") #Filter 7
cut20 = ROOT.TCut(cut21+cut22+cut23)

#### Bestimmen der angeregten Zustände

Wenn Du die Filter festgelegt hast, kannst Du die nächste Zelle ausführen.
Das Programm zeichnet Dir ein Histogramm, in dem die Kombination $\Xi_c^+ K^-$ und die Kombination $\Xi_c^+ K^+$ eingezeichnet sind.
Du sollst dann mithilfe des Cursors die Zustände bestimmen.

In [ ]:
ROOT.gStyle.SetOptStat(0) #Entfernen HistLegende

#Erstellen Hist, Kombinieren Hist
h2 = ROOT.TH1F("h2","Kombination #Xi_{c}^{+} mit K^{#pm}",100,2980,3200)
h3 = ROOT.TH1F("h3","h3",100,2980,3200) #WS
c2 = ROOT.TCanvas("myCanvasName2","Titel2",800,600)
tree1.Draw("lab0_M-lab1_M+2467.89>>h2",cut10+cut20)
tree2.Draw("lab0_M-lab1_M+2467.89>>h3",cut10+cut20) #WS
h2.SetLineWidth(2)
h2.Draw("MIN0")
h3.SetLineColor(2) #WS
h3.SetLineWidth(2) #WS
h3.Draw("same") #WS
%jsroot
c2.Draw()
h2.GetXaxis().SetTitle("m(#Xi_{c}^{+} K^{#pm}) [MeV]")
h2.GetYaxis().SetTitle("Eintr#ddot{a}ge pro 2.2MeV")

leg=ROOT.TLegend(0.85,0.8,0.99,0.99)
leg.AddEntry(h2,"#Xi_{c}^{+} K^{-}","l")
leg.AddEntry(h3,"#Xi_{c}^{+} K^{+}","l") #WS
leg.Draw()
h2.SaveAs("output_gruppe_%d.root"%(gruppe))